In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense
from keras import backend as K
import numpy as np
from keras.preprocessing import image


## Dimension of our Images

In [2]:

img_width,img_height = 150 , 150
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 500
epochs = 50
batch_size = 30

if K.image_data_format() == 'channels_first':
    input_shape = (3,img_width,img_height)
else:
    input_shape = (img_width,img_height,3)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

## This is the augmentation configuration we will use for testing

only rescaling

In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width,img_height),
    batch_size = batch_size,
    class_mode = 'binary')

Found 20000 images belonging to 2 classes.


In [5]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width,img_height),
    batch_size = batch_size,
    class_mode = 'binary')

Found 5000 images belonging to 2 classes.


In [6]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size =(2,2)))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.add(Conv2D(32,(3,3),input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size =(2,2)))



model.add(Conv2D(64,(3,3),input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size =(2,2)))

In [9]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        1

In [11]:
model.compile(loss='binary_crossentropy',optimizer ='rmsprop',metrics = ['accuracy'])

## this is the augmentation configuration we will use for training

In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
66/66 [==============================] - 85s 1s/step - loss: 0.7007 - accuracy: 0.5364 - val_loss: 0.6720 - val_accuracy: 0.5667
Epoch 2/50
66/66 [==============================] - 71s 1s/step - loss: 0.6837 - accuracy: 0.6076 - val_loss: 0.6204 - val_accuracy: 0.6792
Epoch 3/50
66/66 [==============================] - 63s 948ms/step - loss: 0.6512 - accuracy: 0.6394 - val_loss: 0.5957 - val_accuracy: 0.6604
Epoch 4/50
66/66 [==============================] - 62s 936ms/step - loss: 0.6270 - accuracy: 0.6657 - val_loss: 0.5883 - val_accuracy: 0.6708
Epoch 5/50
66/66 [==============================] - 65s 985ms/step - loss: 0.6224 - accuracy: 0.6727 - val_loss: 0.6226 - val_accuracy: 0.6292
Epoch 6/50
66/66 [==============================] - 61s 931ms/step - loss: 0.5693 - accuracy: 0.6934 - val_loss: 0.5358 - val_accuracy: 0.7333
Epoch 7/50
66/66 [==============================] - 68s 1s/step - loss: 

In [13]:
model.save('first_try.h5')

img_pred = image.load_img('gali_dog.jpeg',target_size = (150,150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred,axis = 0)

In [14]:
rslt = model.predict(img_pred)
print(rslt)
if rslt[0][0] == 1:
    print('This Is A Dog')
else:
    print('This Is A cat')
    

[[0.]]
This Is A cat


### Loading Already Made Model


In [1]:
from keras.models import load_model
loaded_model = load_model('first_try.h5')

In [2]:
import numpy as np
from keras.preprocessing import image
img_pred = image.load_img('data/validation/cats/cat.77.jpg',target_size = (150,150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred,axis = 0)
result = loaded_model.predict(img_pred)
print(result)

[[0.]]
